[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/timz815/360-NLP-Project/blob/main/test_model.ipynb)

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen3-4B",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

In [ ]:
from datasets import Dataset
import json, os

jsonl_path = "/content/drive/MyDrive/movie_dialogue.jsonl"
data = [json.loads(line) for line in open(jsonl_path, encoding="utf-8")]

def format_example(ex):
    # Must match the keys in your JSONL
    messages = [
        {"role": "user",      "content": ex["english"]},
        {"role": "assistant", "content": ex["chinese"]},
    ]
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    )
    return {"text": text}

ds = Dataset.from_list(data).map(format_example, remove_columns=data[0].keys())

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    lora_alpha=128,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_dropout=0.0,
    bias="none",
    use_gradient_checkpointing="unsloth",
)

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    train_dataset=ds,
    tokenizer=tokenizer,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=20,
        max_steps=300,          # increase for real run
        logging_steps=1,
        output_dir="/content/drive/MyDrive/qwen3-4b-dialogue-lora",
        optim="adamw_8bit",
        num_train_epochs=1,     # or 2-3
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
    ),
)
trainer.train()
trainer.save_model("/content/drive/MyDrive/qwen3-4b-dialogue-lora/final")

In [ ]:
#QUICK TEST
FastLanguageModel.for_inference(model)

messages = [{"role": "user", "content": "I’m gonna make him an offer he can’t refuse."}]
inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
out = model.generate(input_ids=inputs, max_new_tokens=128, temperature=0.7, do_sample=True)
print(tokenizer.decode(out[0], skip_special_tokens=True))

In [ ]:
model.save_pretrained_gguf(
    "/content/drive/MyDrive/qwen3-4b-dialogue-lora/gguf",
    tokenizer,
    quantization_method="q4_k_m"
)